In [2]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def deriv_sigmoid(x):
    fx = sigmoid(x)
    return fx * (1 - fx)

def mseLoss(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return ((y_true - y_pred) ** 2).mean()

def classify(output):
    return "Female" if output <= 0.5 else "Male"

In [3]:
class NeuralNetwork:
    def __init__(self):
        # 2 -> 2 -> 1
        self.w1 = np.random.normal()
        self.w2 = np.random.normal()
        self.w3 = np.random.normal()
        self.w4 = np.random.normal()
        self.w5 = np.random.normal()
        self.w6 = np.random.normal()

        self.b1 = np.random.normal()
        self.b2 = np.random.normal()
        self.b3 = np.random.normal()

        # scalers
        self.mu = None
        self.sigma = None

    def feedForward(self, x):
        h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
        h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
        o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
        return o1

    def scaleFit_(self, x):
        self.mu = x.mean(axis=0)
        self.sigma = x.std(axis=0) + 1e-8

    def scale_(self, x):
        return (x - self.mu) / self.sigma

    def train(self, inputs, labels):
        self.scaleFit_(inputs)
        inputs = self.scale_(inputs)

        learn_rate = 0.1
        epochs = 1000

        for epoch in range(epochs):
            for x, y_true in zip(inputs, labels):
                # Forward
                sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
                h1 = sigmoid(sum_h1)

                sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
                h2 = sigmoid(sum_h2)

                sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
                o1 = sigmoid(sum_o1)
                y_pred = o1

                # dL/dy_pred
                d_l_d_ypred = -2 * (y_true - y_pred)

                # Output neuron grads
                d_ypred_d_sumo1 = deriv_sigmoid(sum_o1)
                d_ypred_d_w5 = h1 * d_ypred_d_sumo1
                d_ypred_d_w6 = h2 * d_ypred_d_sumo1
                d_ypred_d_b3 = d_ypred_d_sumo1
                d_ypred_d_h1 = self.w5 * d_ypred_d_sumo1
                d_ypred_d_h2 = self.w6 * d_ypred_d_sumo1

                # Hidden neuron h1 grads
                d_h1_d_sumh1 = deriv_sigmoid(sum_h1)
                d_h1_d_w1 = x[0] * d_h1_d_sumh1
                d_h1_d_w2 = x[1] * d_h1_d_sumh1
                d_h1_d_b1 = d_h1_d_sumh1

                # Hidden neuron h2 grads
                d_h2_d_sumh2 = deriv_sigmoid(sum_h2)
                d_h2_d_w3 = x[0] * d_h2_d_sumh2
                d_h2_d_w4 = x[1] * d_h2_d_sumh2
                d_h2_d_b2 = d_h2_d_sumh2

                # Update (gradient descent: param -= lr * gradient)
                # Output layer
                self.w5 -= learn_rate * d_l_d_ypred * d_ypred_d_w5
                self.w6 -= learn_rate * d_l_d_ypred * d_ypred_d_w6
                self.b3 -= learn_rate * d_l_d_ypred * d_ypred_d_b3

                # Hidden layer h1
                self.w1 -= learn_rate * d_l_d_ypred * d_ypred_d_h1 * d_h1_d_w1
                self.w2 -= learn_rate * d_l_d_ypred * d_ypred_d_h1 * d_h1_d_w2
                self.b1 -= learn_rate * d_l_d_ypred * d_ypred_d_h1 * d_h1_d_b1

                # Hidden layer h2
                self.w3 -= learn_rate * d_l_d_ypred * d_ypred_d_h2 * d_h2_d_w3
                self.w4 -= learn_rate * d_l_d_ypred * d_ypred_d_h2 * d_h2_d_w4
                self.b2 -= learn_rate * d_l_d_ypred * d_ypred_d_h2 * d_h2_d_b2

            # loss once per epoch
            if epoch % 50 == 0 or epoch == epochs - 1:
                y_preds = np.apply_along_axis(self.feedForward, 1, inputs)
                print(f"Epoch {epoch:4d} | loss: {mseLoss(labels, y_preds):.4f}")

data = np.array([[84, 176], [80, 157], [75, 160], [95, 190]], dtype=float)
labels = np.array([1, 0, 0, 1], dtype=float)

network = NeuralNetwork()
network.train(data, labels)
print("Network trained")

Epoch    0 | loss: 0.2272
Epoch   50 | loss: 0.1370
Epoch  100 | loss: 0.0739
Epoch  150 | loss: 0.0402
Epoch  200 | loss: 0.0246
Epoch  250 | loss: 0.0168
Epoch  300 | loss: 0.0124
Epoch  350 | loss: 0.0097
Epoch  400 | loss: 0.0078
Epoch  450 | loss: 0.0065
Epoch  500 | loss: 0.0056
Epoch  550 | loss: 0.0049
Epoch  600 | loss: 0.0043
Epoch  650 | loss: 0.0038
Epoch  700 | loss: 0.0035
Epoch  750 | loss: 0.0032
Epoch  800 | loss: 0.0029
Epoch  850 | loss: 0.0027
Epoch  900 | loss: 0.0025
Epoch  950 | loss: 0.0023
Epoch  999 | loss: 0.0022
Network trained


In [4]:
# let's predict
persons = (
    ("Person 01", np.array([63, 165])),
    ("Person 02", np.array([70, 177])),
    ("Person 03", np.array([80, 177]))
)

print("==== Gender Prediction Results ====")
for name, data in persons:
    weight, height = data
    print(f"👤 {name}")
    print(f"Height: {height} cm, Weight: {weight} kg")
    scaled_data = network.scale_(np.array(data, dtype=float))
    pred = network.feedForward(scaled_data)
    print(f"Prediction : {pred:.3f} → {classify(pred)}\n")
print("===================================")


==== Gender Prediction Results ====
👤 Person 01
Height: 165 cm, Weight: 63 kg
Prediction : 0.039 → Female

👤 Person 02
Height: 177 cm, Weight: 70 kg
Prediction : 0.199 → Female

👤 Person 03
Height: 177 cm, Weight: 80 kg
Prediction : 0.867 → Male

